In [1]:
%pip install -q agent-protocol-client

Note: you may need to restart the kernel to use updated packages.


# Python Agent Protocol Client Example

This example was copied and modified from https://agentprotocol.ai/clients/python.

In [2]:
import asyncio

# from agent_protocol.models import StepRequestBody
from agent_protocol_client import (
    Configuration,
    ApiClient,
    StepRequestBody,
    TaskRequestBody,
    AgentApi,
)

In [3]:
# Defining the host is optional and defaults to http://localhost
# See configuration.py for a list of all supported configuration parameters.
configuration = Configuration(host="http://localhost:8000/api")

In [4]:
async def main():
    # Enter a context with an instance of the API client
    async with ApiClient(configuration) as api_client:
        # Create an instance of the API class
        api_instance = AgentApi(api_client)
        task_request_body = TaskRequestBody(input="Write 'Hello world!' to hi.txt.")

        response = await api_instance.create_agent_task(
            task_request_body=task_request_body
        )
        print("The response of AgentApi->create_agent_task:\n")
        print(response)
        print("\n\n")

        task_id = response.task_id
        i = 1

        while (
            step := await api_instance.execute_agent_task_step(
                task_id=task_id, step_request_body=StepRequestBody(input=i)
            )
        ) and step.is_last is False:
            print("The response of AgentApi->execute_agent_task_step:\n")
            print(step)
            print("\n\n")
            i += 1

        print("Agent finished its work!")

        artifacts = await api_instance.list_agent_task_artifacts(task_id=task_id)
        for artifact in artifacts:
            if artifact.file_name == "hi.txt":
                content = await api_instance.download_agent_task_artifact(
                    task_id=task_id, artifact_id=artifact.artifact_id
                )
                print(f'The content of the hi.txt is {content})')
                break
        else:
            print("The agent did not create the hi.txt file.")

# asyncio.run(main())
await main()

ApiException: (501)
Reason: Not Implemented
HTTP response headers: <CIMultiDictProxy('Date': 'Thu, 25 Jul 2024 03:07:37 GMT', 'Server': 'uvicorn', 'Content-Type': 'application/problem+json', 'Content-Length': '242')>
HTTP response body: {"type": "about:blank", "title": "Not Implemented", "detail": "Cannot resolve operationId \"timestep.api.ap.v1.controllers.agent_controller.create_agent_task\"! Import error was \"No module named 'timestep.api.ap.v1.models'\"", "status": 501}
